In [ ]:
# ================================
# Generate Thermodynamic Data for R134a Cycle
# ================================
# Copy-paste into a Python script / Colab cell

!pip install --quiet CoolProp pandas numpy

import numpy as np
import pandas as pd
from CoolProp.CoolProp import PropsSI
import math

# ---- Settings for data grid ----
REF = "R134a"
eta_comp = 0.75  # isentropic efficiency

# Ranges for variables
Te_list = np.linspace(273.15 + 0, 273.15 + 10, 25)         # Evaporator temp: 0°C → 10°C
Tc_list = np.linspace(303.15 + 5, 303.15 + 15, 25)         # Condenser temp: ~35°C → ~45°C
superheat_list = np.linspace(0, 10, 5)                     # Superheat, in K
subcool_list = np.linspace(0, 10, 5)                       # Subcooling, in K

# Prepare storage
data = []

for Te in Te_list:
    for Tc in Tc_list:
        for dT_sh in superheat_list:
            for dT_sc in subcool_list:
                try:
                    # 1. Evaporator state (saturated vapor + superheat)
                    T1 = Te + dT_sh
                    h1 = PropsSI('H', 'T', T1, 'Q', 1, REF)
                    s1 = PropsSI('S', 'T', T1, 'Q', 1, REF)
                    P1 = PropsSI('P', 'T', T1, 'Q', 1, REF)

                    # 2. Condenser state (saturated liquid - subcooling)
                    T3 = Tc - dT_sc
                    h3 = PropsSI('H', 'T', T3, 'Q', 0, REF)
                    P3 = PropsSI('P', 'T', Tc, 'Q', 0, REF)

                    # 3. Compression (isentropic)
                    h2s = PropsSI('H', 'P', P3, 'S', s1, REF)
                    h2 = h1 + (h2s - h1) / eta_comp

                    # 4. Expansion valve (isenthalpic)
                    h4 = h3

                    # 5. Performance metrics
                    Qe = h1 - h4
                    Wc = h2 - h1
                    if Qe <= 0 or Wc <= 0 or math.isnan(Qe) or math.isnan(Wc):
                        COP = np.nan
                    else:
                        COP = Qe / Wc

                    # Save a row
                    data.append({
                        'Te_K': Te,
                        'Tc_K': Tc,
                        'superheat_K': dT_sh,
                        'subcool_K': dT_sc,
                        'h1': h1,
                        'h2s': h2s,
                        'h2': h2,
                        'h3': h3,
                        'Qe': Qe,
                        'Wc': Wc,
                        'COP': COP,
                        'P1': P1,
                        'P3': P3,
                    })

                except Exception as e:
                    # If any state is invalid, store NaN
                    data.append({
                        'Te_K': Te,
                        'Tc_K': Tc,
                        'superheat_K': dT_sh,
                        'subcool_K': dT_sc,
                        'h1': np.nan,
                        'h2s': np.nan,
                        'h2': np.nan,
                        'h3': np.nan,
                        'Qe': np.nan,
                        'Wc': np.nan,
                        'COP': np.nan,
                        'P1': np.nan,
                        'P3': np.nan,
                    })

# Create DataFrame and drop invalid rows
df = pd.DataFrame(data)
df = df.dropna(subset=['COP'])  # drop rows where COP is NaN

# Save to CSV
df.to_csv("r134a_cycle_data.csv", index=False)
print("Generated data shape:", df.shape)
print(df.head())
from google.colab import files
files.download("r134a_cycle_data.csv")



Generated data shape: (15625, 13)
     Te_K    Tc_K  superheat_K  subcool_K             h1            h2s  \
0  273.15  308.15          0.0        0.0  398603.453628  421623.557134   
1  273.15  308.15          0.0        2.5  398603.453628  421623.557134   
2  273.15  308.15          0.0        5.0  398603.453628  421623.557134   
3  273.15  308.15          0.0        7.5  398603.453628  421623.557134   
4  273.15  308.15          0.0       10.0  398603.453628  421623.557134   

              h2             h3             Qe            Wc       COP  \
0  429296.924969  249006.651856  149596.801772  30693.471342  4.873896   
1  429296.924969  245350.440950  153253.012678  30693.471342  4.993017   
2  429296.924969  241722.392309  156881.061319  30693.471342  5.111219   
3  429296.924969  238121.229641  160482.223986  30693.471342  5.228546   
4  429296.924969  234545.776695  164057.676933  30693.471342  5.345035   

              P1             P3  
0  292803.182339  886980.983608  
1 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>